![RDD key pair](media/18.sparkml.jpg)

# 10 Spark ML: Machine Learning Library

Librería de algoritmos paralelos de ML para datos masivos

-   Algoritmos clásicos de machine learning: clasificación, regresión, clustering, filtrado colaborativo
-   Otros algoritmos: extracción de características, transformación, reducción de dimensionalidad y selección
-   Herramientas para construir, evaluar y ajustar pipelines de ML
-   Otras utilidades: álgebra lineal, estadística, manejo de datos, etc.


Dos paquetes:

-   **spark.mllib**: API original, basada en RDDs
-   **spark.ml**: API de alto nivel, basada en DataFrames

Documentación:
[spark.apache.org/docs/latest/mllib-guide.html](http://spark.apache.org/docs/latest/mllib-guide.html)
y
[spark.apache.org/docs/latest/ml-guide.html](http://spark.apache.org/docs/latest/ml-guide.html)

In [1]:
!pip install pyspark

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Create apache spark context
from pyspark import SparkContext
sc = SparkContext(master="local", appName="Mi app")

In [10]:
# Stop apache spark context
sc.stop()

#### Ejemplo

Usa el algoritmo de clustering [KMeans](http://spark.apache.org/docs/latest/mllib-clustering.html#k-means) para agrupar datos de vectores dispersos en dos clusters.

In [1]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector
import numpy as np

# Define un array de 4 vectores dispersos, de 3 elementos cada uno
sparse_data = [
     SparseVector(3, {1: 1.2}),
     SparseVector(3, {1: 1.1}),
     SparseVector(3, {0: 0.9, 2: 1.0}),
     SparseVector(3, {0: 1.0, 2: 1.1})
 ]

for i in range(4):
    print(sparse_data[i].toArray())

[0.  1.2 0. ]
[0.  1.1 0. ]
[0.9 0.  1. ]
[1.  0.  1.1]


In [4]:
# Construye el modelo (agrupa los datos en 2 clusters)
model = KMeans.train(sc.parallelize(sparse_data), \
                     2, initializationMode="k-means||",\
                     seed=50, initializationSteps=5, \
                     epsilon=1e-4)

print("Centros de los clusters: {0}".format(model.clusterCenters))

for i in range(4):
    print("Cluster para el nodo {0} = {1}"
           .format(i, model.predict(sparse_data[i])))

punto = SparseVector(3, {0: 0.9, 1:1.0, 2: 1.0})
print("\nCluster para el nodo {0} = {1}".format(punto, model.predict(punto)))

Centros de los clusters: [array([0.  , 1.15, 0.  ]), array([0.95, 0.  , 1.05])]
Cluster para el nodo 0 = 0
Cluster para el nodo 1 = 0
Cluster para el nodo 2 = 1
Cluster para el nodo 3 = 1

Cluster para el nodo (3,[0,1,2],[0.9,1.0,1.0]) = 1


In [5]:
# Salva el modelo en un directorio temporal
import os, tempfile
path = tempfile.mkdtemp()
model.save(sc, path)

# Vuelve a cargar el modelo
sameModel = KMeansModel.load(sc, path)

for i in range(4):
    print(sameModel.predict(sparse_data[i]) == model.predict(sparse_data[i]))

print(sameModel.predict(punto) == model.predict(punto))

True
True
True
True
True


In [6]:
# Borra el directorio temporal
from shutil import rmtree
try:
     rmtree(path)
except OSError:
     pass